## Assignment - 8
In PyQuil, implement the Deutsch-Jozsa algorithm and the Bernstein-Vazirani algorithm.  Write detailed comments in the code about why it works.  Run the programs on the simulator.  Write a report that covers the following three points.

1. Design

    * Present the design of how you implemented the black-box function U_f.  Assess how visually neat and easy to read it is.
    * Present the design for how you prevent the user of U_f from accessing the implementation of U_f.  Assess how well you succeeded.
    * Present the design of how you parameterized the solution in n.
    * Discuss the number of lines and percentage of code that your two programs share.  Assess how well you succeeded in reusing code from one program to the next.


2. Evaluation

    * Discuss your effort to test the two programs and present results from the testing.  Discuss whether different cases of U_f lead to different execution times.
    * What is your experience with scalability as n grows?  Present a diagram that maps n to execution time.

3. Instructions

    * Present a README file that describes how to input the function f, how to run the program, and how to understand the output.
    * Submit three files, one for each program and one with the report.

In [3]:
import numpy as np

#sample balanced function designed for N =2 for testing the code
def f(x):
	# print(x, type(x))
	if(x =="01" or x=="10"):
		return 1
	else:
		return 0
    
#sample constant function designed for N =2 for testing the code
def f_1(x):
	return 1

#sample constant function designed for N =2 for testing the code
def f_2(x):
	return 0


#this function creates the blackbox oracle Uf matrix for a given function f that is parameterised for input size n
def createUf(f,n):
    
    #this dictionary represents the correspondence between bit combinations and Uf indices
	indices_dict = {}
	counter = 0
	Uf = np.zeros((2**(n+1),2**(n+1)))

	bfx = 0

    # the for loops given below form the different pattern combinations that we need. The mapping between these combination and Uf matrix indices is stored in indices_dict
	for i in range(2**(n+1)):
		binary_form = ('{0:0'+str(n+1)+'b}').format(i) 
		indices_dict[binary_form] = counter
		counter += 1


    #we then iterate through this pattern dictionary
	for key,val in indices_dict.items():
        
        #input to the function is the first n bits of the elements (bit patterns) from the dictionary
		x = key[0:n]
        
        #fx represents the output of function f given the input x
		fx = str(f(x))
        
        #b is the last bit of the bit pattern in the dictionary item
		b = key[n]
        
        #below we have the (f(x) + b) mod 2
		if(b==fx):
			bfx = '0'
		else:
			bfx = '1'
			# print(bfx)
        
        #the final bit string is the concatenation of the input x and bfx
		target = x + bfx
		# print('t',target)
        
        #using indices_dict we can now find the index that corresponds to this output
		target_index = indices_dict[target]
		# print(val,target_index)
        
        #now using the target indiex we can create a bit pattern with all 0s and 1 at the target index position
		Uf[val][target_index] = 1
	return Uf




In [4]:

'''
Takes a qubitString as input and returns a matrix that represents the qubit in the standard basis.
Ex: |00> = [1,0,0,0]
'''
def getQubitVector(bitString):
    bitDict = {}
    bitDict['0'] = np.array([1 , 0])
    bitDict['1'] = np.array([0 , 1])
    matrix = [1]
    
    for c in bitString:
        matrix = np.kron(matrix,bitDict[c])
    return matrix

'''
Given a value of n, returns all possible bit-strings of size n.
This function will return a list of 2^n bit strings
'''
def getAllPossibleNBitStrings(n):
    if n==1:
        return ['0','1']
    
    children = getAllPossibleNBitStrings(n-1)
    result = []
    for i in children:
        result.append('0'+i)
        result.append('1'+i)
    return result

'''
Given the function to execute, and the qubits |x1x2..xn> and the ancilla|b>.
This function will return a bit vector representing |x1x2..xn> |b+f(x)>
'''
def getFunctionResult(function,x,ancilla):
    fx = function(x)
    newAncilla = str((((int)(ancilla) + fx)%2))
    result = x+newAncilla
    return getQubitVector(result)

'''
Given a function and n qubits. This function will verify if the Uf matrix 
generated for this function matches the expected output. 
i.e., it checks if Uf|x1x2..xn>|b> = |x1x2..xn> |b+f(x)>
'''
def verifyUf(function,n,debug=False):
    UfMatrix = createUf(function,n)   
    if debug:
        print("\nUfMatrix for function \n"+str(UfMatrix)+"\n")
        
    inputs = getAllPossibleNBitStrings(n+1)
    valid = True
    
    for i in inputs:
        inputBitVector = getQubitVector(i)
        
        if debug:
            print("Input BitString {}".format(i))
            print("Input BitVector {}\n".format(inputBitVector))
        
        functionOutput = getFunctionResult(function,i[0:n],i[n])
        UfOutput = np.matmul(UfMatrix,inputBitVector)
        
        if debug:
            print("\tUf Output {}".format(UfOutput))
            print("\t F Output {}\n".format(functionOutput))
        
        if np.array_equal(UfOutput,functionOutput) is not True:
            print("ERROR FOR {}\n".format(i))
            valid=False
            
    return valid


print("\nVerifying F1 constant\n")
result = verifyUf(f_1,2)
print("F_1 constant verification result {}\n".format(result))
print("\nVerifying F2 constant\n")
result = verifyUf(f_2,2)
print("F_2 constant verification result {}\n".format(result))
print("\nVerifying F balanced\n")
result = verifyUf(f,2)
print("F balanced verification result {}\n".format(result))


Verifying F1 constant

F_1 constant verification result True


Verifying F2 constant

F_2 constant verification result True


Verifying F balanced

F balanced verification result True



In [8]:
from pyquil import Program
from pyquil.gates import *
from pyquil import get_qc
from pyquil.quilatom import unpack_qubit
from pyquil.quil import DefGate

    
    
def runMainCircuit(function,n,nTrials,debug=False):
    p = Program()

    #create a matrix representing Uf
    UfMatrix = createUf(function,n)
    
    #for a n bit function, we need n+1 qubits(one ancilla bit)
    n = n+1
    
    qc_name = "{}q-qvm".format(n)
    qc = get_qc(qc_name)

    #setting last qubit to 1
    p += X(n-1)
    
    #adding Hadamard gates to all qubits
    for i in range(0,n):
        p += H(i)

    GateName = "UF_GATE"

    #create a gate that uses the Uf matrix and pass all qubits to this as input 
    uf_gate_definition = DefGate(GateName, UfMatrix)
    qubits = [unpack_qubit(i) for i in range(0,n)]

    #adding Uf gate
    p+=Program(uf_gate_definition,Gate(name=GateName, params=[],qubits=qubits))

    for i in range(0,n-1):
        p += H(i)

    if debug:
        print(p)
        
    results = qc.run_and_measure(p, trials=nTrials)
    return results

### Deutsch-Jozsa

In [9]:
nTrials = 1

def isBalancedOrConstant(function,nQubits):
    results = runMainCircuit(function,nQubits,nTrials)
    constant = True
    for i in range(0,nQubits):
        if results[i][0] != 0:
            constant = False
            break
    
    if constant:
        return "CONSTANT"
    else:
        return "BALANCED"

In [10]:
print(isBalancedOrConstant(f,2))
print(isBalancedOrConstant(f_1,2))
print(isBalancedOrConstant(f_2,2))

BALANCED
CONSTANT
CONSTANT


### Bernstein-Vazirani

In [11]:
nTrials = 1

def constructFunction(function,nQubits):
    b = function("0"*nQubits)
    a = ""
    results = runMainCircuit(function,nQubits,nTrials)
    for i in range(0,nQubits):
        a+=str(results[i][0])
    return a,b

In [12]:
print(constructFunction(f,2))

('11', 0)
